In [1]:
#TODO update comments on relevant blocks to reflect changes to inter-annual seasons handling (now handled in NDVI timeseries computations, instead of the temporal anomaly section)

#imports, global consts, inits
import ee
import geemap

#Mostly used when testing something with geemap. You (probably) won't see them used in the code bellow.
NDVI_VIZ_PARAMS = {"min" : -1, "max": 1, "palette" : ["blue", "white", "green"]}
NDVI_VIZ_PARAMS_CROP_ONLY = {"min" : 0.1, "max": 0.5, "palette" : ["red", "yellow", "green"]}

ee.Initialize(project='seamproj01')
map = geemap.Map()

sudanStateBorders = ee.FeatureCollection("projects/seamproj01/assets/SudanStateBorders")    #Shapefiles for Sudan administration borders, via OCHA HDX. This dataset is used for 
                                                                                        #state/admin division clipping (offline, on QGIS) of the cropland masks bellow. Mostly used
                                                                                        #when testing with geemap.

testArea =  ee.FeatureCollection("projects/seamproj01/assets/test_area") #A 55km2 block in the Gezira state east of the Blue Nile.

khartoumCropland = ee.FeatureCollection("projects/seamproj01/assets/khartoum_cropmask_v3")  #From the Copernicus Moderate Dynamic Land Cover dataset. Extracted cropland pixels, then clipped
                                                                                                    #to Khartoum state. Polygons less than 0.3km2 in area are removed. Holes smaller than 0.5km2 are filled.
                                                                                                    #Subdivided using level 2 OCHA admin subdivions (with Karrari and Um Bada merged into one)
geziraCropland = ee.FeatureCollection("projects/seamproj01/assets/gezira_cropmask_v3")   #Also based on Copernicus MDLC, cliped first with GlobCover dataset (via FAO LCLU). Then
                                                                                                        #Seperated based on position relative to Blue Nile (east or west)

In [2]:
def ProcessMODISCollection( col : ee.ImageCollection,
                            roi : ee.Geometry,
                            startDate : str,
                            endDate : str) -> ee.ImageCollection:
    
    #mappable functions (leaving them scoped inside the MODIS function because similarily named ones with different implementation exist for Sentinel as well)
    def MaskPoortQualityPixels(img : ee.Image) -> ee.Image:
        qaBand = img.select("State")
        #TODO add snow/ice masking
        mask = qaBand.bitwiseAnd(3).eq(0) #pixel is clear from cloud (bits 0 and 1)
        mask = mask.And(qaBand.bitwiseAnd(4).eq(0)) #not cloud shadow (bit 2)
        mask = mask.And(qaBand.bitwiseAnd(768).eq(0)) #no cirrus (bits 8 and 9)
        return img.updateMask(mask)
    
    output = col.filterDate(startDate, endDate).filterBounds(roi)
    output = output.map(lambda image : image.clip(roi).copyProperties(image, image.propertyNames()))
    output = output.map(MaskPoortQualityPixels)
    output = output.map(lambda image : image.normalizedDifference(["sur_refl_b02", "sur_refl_b01"]).rename("NDVI").copyProperties(image, image.propertyNames()))
    return output


def ProcessSentinelCollection(  col : ee.ImageCollection,
                                roi : ee.Geometry,
                                startDate : str,
                                endDate : str,
                                targetMonths : list) -> ee.ImageCollection:
    
    def MaskPoortQualityPixels(img : ee.Image) -> ee.Image:
        qaBand = img.select("SCL")
        #Unlike MODIS, the quality band used here, "SCL," contains a single int meant to be interpreted as a single int.
        #https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/scene-classification/
        
        mask = qaBand.eq(3) #cloud shadows
        mask = mask.And(qaBand.eq(6)) #water
        mask = mask.And(qaBand.eq(8)) #medium probability clouds
        mask = mask.And(qaBand.eq(9)) #high probability clouds
        mask = mask.And(qaBand.eq(10)) #thin cirrus
        mask = mask.And(qaBand.eq(11)) #snow/ice

        mask = mask.neq(1) #flip so it masks out the above
        
        return img.updateMask(mask)
    
    #to try and improve performance a bit, we'll limit the collection to only target months, using ee.Filter.calendarRange(start, end, field)
    #problem is, targetMonths may no be in order (consider inter-annual years), so we can't just take first and last entry. can't take max or min either else
    #it would defeat purpose (min/max of [11, 12, 1, 2] would result in twelve months).
    periodStart = 0
    periodEnd = 0
    periods = []
    for i in range (1, len(targetMonths)):
        if (targetMonths[i] > targetMonths[i - 1]):
            periodEnd = i
        else:
            periods.append([targetMonths[periodStart], targetMonths[periodEnd]])
            periodStart = periodEnd = i

    periods.append([targetMonths[periodStart], targetMonths[periodEnd]])

    monthsFilter = ee.Filter.calendarRange(periods[0][0], periods[0][1], "month")
    for i in range (1, len(periods)):
        monthsFilter = ee.Filter.Or(monthsFilter, ee.Filter.calendarRange(periods[i][0], periods[i][1], "month"))
        
    #filter then process the output collection
    output = col.filterDate(startDate, endDate).filterBounds(roi).filter(monthsFilter)
    output = output.select("B4", "B8", "SCL") #grasping at straws trying to make this thing run faster... TODO experiment to see if this actually has an effect
    output = output.map(lambda image : image.clip(roi).copyProperties(image, image.propertyNames()))
    output = output.map(MaskPoortQualityPixels)
    output = output.map(lambda image : image.normalizedDifference(["B8", "B4"]).rename("NDVI").copyProperties(image, image.propertyNames()))
    return output


In [3]:
#roi = testArea
roi = geziraCropland
#roi = khartoumCropland

#This it the name of the property (or "attribute", in GIS-speak) that contains the identifier for the subdivision of the roi used as climate divisions for spatial anomaly
#analysis, and also used in splitting rasters of the output.
subdivisionPropertyName = "NAME"

#the output raster will be prefixed with this
#projectName = "TestProj" 
projectName = "Gezira" 
#projectName = "Khartoum"

#spatial resolution of the output file(s), in meters
targetOutputScale = 30

#time-series limits. All inclusive.
yearStart = 2019
monthStart = 1
yearEnd = 2024
monthEnd = 2

#targetMonths are the months comprising the season for analysis.
#WARNING! MUST BE 4 Values! Otherwise, the Temporal Anomaly Analysis component must be adjusted
#WARNING! ORDER OF MONTHS MUST BE CHRONOLOGICAL! If the season is inter-annual, start with the months in the first year, then the second year
#e.g. if the season starts on November, the list would be [11, 12, 1, 2]
#targetMonths = [7, 8, 9, 10] #"Summer" season in Sudan (technically Autumn). This covers the growth periods of crops such as sorghum.
targetMonths = [11, 12, 1, 2] #"Winter" season in Sudan. This covers growth periods of crops such as wheat

dateStart = f"{yearStart}-{monthStart}-1"
dateEnd = f"{yearEnd}-{monthEnd + 1}-1" if monthEnd < 12 else  f"{yearEnd + 1}-1-1"

# modis = ee.ImageCollection("MODIS/061/MOD09Q1")
# modis = ProcessMODISCollection(modis, roi.geometry(), dateStart, dateEnd)

sentinel2SR = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
sentinel2SR = ProcessSentinelCollection(sentinel2SR, roi.geometry(), dateStart, dateEnd, targetMonths)
#TODO the Sentinel-2 L2A data used here miss some dates prior to dec 2018 for Gezira region. This is specific to GEE's version of 2A. Actual 2A on Copernicus Dataspace do cover this period.
#Note that GEE's L1C (the TOA version) does cover this period as well. So consider implementing a TOA-to-SR algo to augment the 2A data where it's lacking.
#Also note: feeding this algorithm months (within range dateStart to dateEnd) without rasters will throw exceptions.

#Set ndviTS to the (processed) dataset you are using.
#ndviTS = modis
ndviTS = sentinel2SR

#Note: ndviTS is only used in computation of the max/range timeseries, but it's not considered in some reduceRegions and exports bellow, which require the scale of the image as argument.
#TODO adjust the reduceRegions and Exports bellow to dynamically use the scale based on that of the selected dataset

In [4]:
# #Checking that enough data are available for sentinel-2 L2C for the selected roi/dates.
# #note: this check does not use the processed data. It tests the original dataset after filtering for roi and date, to speed the process up as ProcessSentinelCollection() can
# #take a significant of time, depending on roi.
# periodStart = 0
# periodEnd = 0
# periods = []
# for i in range (1, len(targetMonths)):
#     if (targetMonths[i] > targetMonths[i - 1]):
#         periodEnd = i
#     else:
#         periods.append([targetMonths[periodStart], targetMonths[periodEnd]])
#         periodStart = periodEnd = i

# periods.append([targetMonths[periodStart], targetMonths[periodEnd]])

# monthsFilter = ee.Filter.calendarRange(periods[0][0], periods[0][1], "month")
# for i in range (1, len(periods)):
#     monthsFilter = ee.Filter.Or(monthsFilter, ee.Filter.calendarRange(periods[i][0], periods[i][1], "month"))
    
# for year in range(yearStart, yearEnd + 1):
#     for month in targetMonths:
#         subColStart = f"{year}-{month}-1"
#         subColEnd = f"{year}-{month + 1}-1" if month < 12 else f"{year + 1}-1-1"
#         ndviRastersForMonth = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterBounds(roi.geometry()).filterDate(subColStart, subColEnd).size().getInfo()

#         print (f"{year}-{month} : available scenes = {ndviRastersForMonth}")

In [5]:
#generting the timeseries (monthly max NDVI and NDVI range) and the statistics for the "pure crop" signal.
#Outputs of this block are two ImageCollections: "timeseries," and "pureCropNDVI"
    #timseries contains images for each season.
        #Each Image has number of bands equal to twice the number of targetMonths (2x4 = 8).
        #Each Image has a property "year" for the season's year, and property "isComplete" showing whether available data covers all targetMonths (1 = true, 0 = false)
        #Each band is named "month_x_max" or "month_x_range", where x is the month's number; "max" and "range" denote whether it encodes the maximum monthly ndvi or ndvi monthly range.
    #pureCropNDVI contains images for each month in the targetMonths (total = 4)
        #each image has 4 bands: "month_x_max_mean" or "month_x_max_stdDev" (and similarily for range).
        #each image has a property "month" with month's number

timeSeries = ee.List([])

lastCompleteSeason = yearStart

for year in range(yearStart, yearEnd + 1):
    
    yearTS = ee.List([])
    isCompleteSeason = True
    
    lastMonth = 0
    calendarYear = year
    lastCompleteSeason = year

    for month in targetMonths:
        if (month < lastMonth):
            calendarYear += 1
        lastMonth = month
        
        if ((calendarYear * 100) + month > (yearEnd * 100) + monthEnd):
            isCompleteSeason = False
            break
        
        
        subPeriodStart = f"{calendarYear}-{month}-1"
        subPeriodEnd = f"{calendarYear}-{month + 1}-1" if month < 12 else  f"{calendarYear + 1}-1-1"
        subCol = ndviTS.filterDate(subPeriodStart, subPeriodEnd)
        
        minMaxNDVI = subCol.reduce(ee.Reducer.minMax())

        monthMax = minMaxNDVI.select("NDVI_max").rename("max")
        monthRange = minMaxNDVI.select("NDVI_max").subtract(minMaxNDVI.select("NDVI_min")).rename("range")
        yearTS = yearTS.add(ee.Image([monthMax, monthRange]).set({"system:index" : f"month_{month}"}))
    
    if(not isCompleteSeason):
        lastCompleteSeason += -1
        break

    yearTS = ee.ImageCollection(yearTS).toBands().set({"year" : year})
    timeSeries = timeSeries.add(yearTS)

timeSeries = ee.ImageCollection(timeSeries)

##uncomment this part to export the ndviTS. Not recommended for Sentinel 2 data for large areas...
##to reduce output size, the data is scaled by 100 the rounded off, and stored as an 8bit int which range 0 to 255, but because ndvi ts ranges from 0 to 1 (zero because we masked out water), the
##resulting raster would range from 0 to 100. Also note the reduced precision this introduces (shouldn't matter much anyway)
##TODO adjust code so bands would have name of year.
##TODO seperate output images for this the same as you do fallowTS bellow.
# task = ee.batch.Export.image.toDrive(
#     image = timeSeries.toBands().multiply(ee.Image(ee.Number(100))).toByte()
#     description = f"{projectName}_ndviTS_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}",
#     #folder='ee_export',
#     region = roi.geometry(),
#     scale = targetOutputScale,
#     crs = 'EPSG:4326',
#     fileFormat = 'GeoTIFF',
#     formatOptions = {
#         'noData': -9999
# })
# task.start()


pureCropNDVI = ee.List([])

for month in targetMonths:
    prefix = f"month_{month}_"
    thisMonthTS = timeSeries.select([prefix + "max", prefix + "range"])

    monthMedian = thisMonthTS.reduce(ee.Reducer.median()).rename([prefix + "max", prefix + "range"]).set({"month" : month})
    
    meanStdReducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs = True)

    monthPureCropNDVI = thisMonthTS.map(lambda img : img.updateMask(img.gt(monthMedian)))
    monthPureCropNDVI = monthPureCropNDVI.reduce(meanStdReducer).set("month", month)

    pureCropNDVI = pureCropNDVI.add(monthPureCropNDVI)

pureCropNDVI = ee.ImageCollection(pureCropNDVI)

In [6]:
#This block computes the temporal anomalies for each season. The output is an ImageCollection called temporalAnomalies, containing Images representing each season.
    #Each Image has number of bands equal to twice the number of targetMonths (2x4 = 8).
    #Each Image has a property "year" for the season's year.
    #Each band is named "month_x_max" or "month_x_range".
#note: The loop handles inter-annual years (assuming the targetMonth list was set correctly). However, the output of the anomalies will be given a "year" property based on its season, not its
#calendar year. For example, for a season that starts on Nov 2013 to Feb 2014, the months of Jan and Feb would be bands in the image with "year" = 2013.

temporalAnomalies = ee.List([])

for year in range (yearStart, lastCompleteSeason + 1):
    seasonAnomalies = ee.List([])
    seasonSeries = timeSeries.filter(ee.Filter.eq("year", year)).first()

    for month in targetMonths:
        prefix = f"month_{month}_"
        pureCropSignal = pureCropNDVI.filter(ee.Filter.eq("month", month)).first()

        taMax = seasonSeries.select(prefix + "max").subtract(pureCropSignal.select(prefix + "max_mean")).divide(pureCropSignal.select(prefix + "max_stdDev")).rename("max")
        taRange = seasonSeries.select(prefix + "range").subtract(pureCropSignal.select(prefix + "range_mean")).divide(pureCropSignal.select(prefix + "range_stdDev")).rename("range")

        ta = ee.Image([taMax, taRange]).set({"system:index" : f"month_{month}"})
        seasonAnomalies = seasonAnomalies.add(ta)

    #if (not incompleteYear):
    seasonAnomalies = ee.ImageCollection(seasonAnomalies).toBands().set({"year" : year})
    temporalAnomalies = temporalAnomalies.add(seasonAnomalies)

temporalAnomalies = ee.ImageCollection(temporalAnomalies)

In [7]:
#Temporal analysis component

def TemporalAnomalyAnalysis(image : ee.Image) -> ee.Image: #to be mapped over temporalAnomalies collection
    taMax = []
    taRange = []

    for month in targetMonths:
        name = f"month_{month}"
        taMax.append(image.select(name + "_max").rename(name))
        taRange.append(image.select(name + "_range").rename(name))
        
    isFallow_1 = taMax[0].lt(-3).And(taMax[1].lt(-3)).And(taMax[2].lt(-3))
    isFallow_1 = isFallow_1.Or(taMax[1].lt(-3).And(taMax[2].lt(-3)).And(taMax[3].lt(-3)))
    isFallow_1 = isFallow_1.rename("max")

    isFallow_2 = taRange[0].lt(-3).And(taRange[1].lt(-3)).And(taRange[2].lt(-3))
    isFallow_2 = isFallow_2.Or(taRange[1].lt(-3).And(taRange[2].lt(-3)).And(taRange[3].lt(-3)))
    isFallow_2 = isFallow_2.rename("range")

    return ee.Image([isFallow_1, isFallow_2]).set({"year" : image.get("year")})

#isFallow_TA the two questions for the temporal anomalies
isFallow_TA = temporalAnomalies.map(TemporalAnomalyAnalysis)

In [8]:
zones = roi.toList(roi.size())
zonesSize = zones.size().getInfo()

print(f"Processing spatial anomalies for {zonesSize} zone(s)") #test

def SpatialAnomalyAnalysis(image : ee.Image) -> ee.Image:
    maxOfMax = ee.List([])
    maxOfRange = ee.List([])
    
    spatialMedianMax = ee.List([])
    spatialMedianRange = ee.List([])
    
    for month in targetMonths:
        name = f"month_{month}"
        _maxOfMax = image.select(name + "_max").rename("max")
        _maxOfRange = image.select(name + "_range").rename("range")
        
        maxOfMax = maxOfMax.add(_maxOfMax)
        maxOfRange = maxOfRange.add(_maxOfRange)

        for i in  range(0, zonesSize):
            targetZone = ee.Feature(zones.get(i)).geometry()
            zoneMaxMedian =  _maxOfMax.reduceRegion(   reducer = ee.Reducer.median(),
                                                        geometry = targetZone,
                                                        tileScale = 2,
                                                        scale = 90, #TODO this was increased to 90 in the original, single zone implementation. Test setting it back.
                                                        maxPixels = 5000000, #TODO same as above (reduced from default 10mil)
                                                        bestEffort = True,
                                                        crs='EPSG:4326',
                                                        ).getNumber("max")
            
            zoneRangeMedian =  _maxOfRange.reduceRegion(    reducer = ee.Reducer.median(),
                                                            geometry = targetZone,
                                                            tileScale = 2,
                                                            scale = 90,
                                                            maxPixels = 5000000,
                                                            bestEffort = True,
                                                            crs='EPSG:4326',
                                                            ).getNumber("range")
            
            zoneMaxMedian = ee.Image(ee.Number(zoneMaxMedian)).clip(targetZone)
            zoneRangeMedian = ee.Image(ee.Number(zoneRangeMedian)).clip(targetZone)

            spatialMedianMax = spatialMedianMax.add(zoneMaxMedian)
            spatialMedianRange = spatialMedianRange.add(zoneRangeMedian)
        
        spatialMedianMax = ee.ImageCollection(spatialMedianMax).mosaic()
        spatialMedianRange = ee.ImageCollection(spatialMedianRange).mosaic()

    maxOfMax = ee.ImageCollection(maxOfMax)
    maxOfRange = ee.ImageCollection(maxOfRange)
    
    isFallow_3 = maxOfMax.reduce(ee.Reducer.max()).lt(spatialMedianMax.multiply(ee.Number(0.8))).set({"year" : year}).rename("max")
    isFallow_4 = maxOfRange.reduce(ee.Reducer.max()).lt(spatialMedianRange.multiply(ee.Number(0.8))).set({"year" : year}).rename("range")

    return ee.Image([isFallow_3, isFallow_4]).set({"year" : image.get("year")})


#isFallow_SA the two questions for the spatial anomalies
isFallow_SA = timeSeries.map(SpatialAnomalyAnalysis)

Processing spatial anomalies for 2 zone(s)


In [9]:
# # #backup - working code

# def SpatialAnomalyAnalysis(image : ee.Image) -> ee.Image:
#     #image has max and range for each month in a year (12 bands). month_x_max, month_x_range, x = month number
#     maxOfMax = ee.List([])
#     maxOfRange = ee.List([])
    
#     spatialMedianMax = ee.List([])
#     spatialMedianRange = ee.List([])

#     for month in targetMonths:
#         name = f"month_{month}"
#         _maxOfMax = image.select(name + "_max").rename("max")
#         _maxOfRange = image.select(name + "_range").rename("range")
        
#         maxOfMax = maxOfMax.add(_maxOfMax)
#         maxOfRange = maxOfRange.add(_maxOfRange)

#         spatialMedianMax =      spatialMedianMax.add(   _maxOfMax.reduceRegion(
#                                                         reducer = ee.Reducer.median(),
#                                                         geometry = roi,
#                                                         tileScale = 2,
#                                                         scale = 90, #doing the analysis for Gezira with Sentinel-2 at even 30 (without bestEffort) hits memory limits.
#                                                         maxPixels = 5000000, #reduced from default 10mil in an attempt to speed up processing of sentinel data.
#                                                         bestEffort = True,
#                                                         crs='EPSG:4326',
#                                                         ).getNumber("max"))
        
#         spatialMedianRange =   spatialMedianRange.add(  _maxOfRange.reduceRegion(
#                                                         reducer = ee.Reducer.median(),
#                                                         geometry = roi,
#                                                         tileScale = 2,
#                                                         scale = 90,
#                                                         maxPixels = 5000000,
#                                                         bestEffort = True,
#                                                         crs='EPSG:4326',
#                                                         ).getNumber("range"))


#     maxOfMax = ee.ImageCollection(maxOfMax)
#     maxOfRange = ee.ImageCollection(maxOfRange)

#     spatialMedianMax = ee.Number(spatialMedianMax.reduce(ee.Reducer.max()))
#     spatialMedianRange = ee.Number(spatialMedianRange.reduce(ee.Reducer.max()))

#     isFallow_3 = maxOfMax.reduce(ee.Reducer.max()).lt(ee.Number(0.8).multiply(spatialMedianMax)).set({"year" : year}).rename("max")
#     isFallow_4 = maxOfRange.reduce(ee.Reducer.max()).lt(ee.Number(0.8).multiply(spatialMedianRange)).set({"year" : year}).rename("range")

#     return ee.Image([isFallow_3, isFallow_4]).set({"year" : image.get("year")})

# #isFallow_SA the two questions for the spatial anomalies
# isFallow_SA = timeSeries.filter(ee.Filter.eq("isComplete", 1)).map(SpatialAnomalyAnalysis)

In [10]:
#Final analysis

isFallowComponents = ee.List([])
for year in range (yearStart, yearEnd + 1):
    if ((year * 100) + targetMonths[0] > (yearEnd * 100) + monthEnd):
            break
    
    yearComponents = ee.List([]).add(isFallow_TA.filter(ee.Filter.eq("year", year)).first().set({"system:index" : f"{year}_TA"}))
    yearComponents = yearComponents.add(isFallow_SA.filter(ee.Filter.eq("year", year)).first().set({"system:index" : f"{year}_SA"}))
    isFallowComponents = isFallowComponents.add(ee.ImageCollection(yearComponents).toBands().set({"year" : year}))

isFallowComponents = ee.ImageCollection(isFallowComponents)

fallowTS = isFallowComponents.map(lambda img : img.reduce(ee.Reducer.sum()).gte(ee.Number(2)).rename(ee.String(img.get("year")))).toBands()

In [11]:
#export

noDataValue = -9999

#split raster into multiple ones based on zone, then export
for i in range (0, zonesSize):
    targetZone = ee.Feature(zones.get(i))
    exportName = f"{projectName}_{targetZone.get(subdivisionPropertyName).getInfo()}_fallow_ts_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}"
    print (f"{i} - exporting file: {exportName}")
    
    task = ee.batch.Export.image.toDrive(
    image = fallowTS.clip(targetZone.geometry()),
    description = exportName,
    region = targetZone.geometry(),
    scale = targetOutputScale,
    crs = 'EPSG:4326',
    fileFormat = 'GeoTIFF',
    formatOptions = {'noData': noDataValue})

    task.start()



# task = ee.batch.Export.image.toDrive(
#     image = fallowTS,
#     description = f"{projectName}_fallow_ts_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}",
#     #folder = 'ee_export',
#     region = roi,
#     scale = targetOutputScale,
#     crs = 'EPSG:4326',
#     fileFormat = 'GeoTIFF',
#     formatOptions = {
#         'noData': noDataValue
#     })

# task.start()

# isFallowComponentsCollapsed = isFallowComponents.toBands()

# task = ee.batch.Export.image.toDrive(
# image = isFallowComponentsCollapsed,
# description = f"{projectName}_fallow_components_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[1]}-{targetMonths[2]}-{targetMonths[3]}",
# #folder = 'ee_export',
# region = roi,
# scal = targetOutputScale,
# crs = 'EPSG:4326',
# fileFormat = 'GeoTIFF',
# formatOptions = {
#     'noData': noDataValue
# })

#task.start()

0 - exporting file: Gezira_Eastern_Gezira_fallow_ts_2019-2024_season_11-12-1-2
1 - exporting file: Gezira_Western_Gezira_fallow_ts_2019-2024_season_11-12-1-2


In [12]:
# map.centerObject(roi)
# map.addLayer(fallowTS, name="FallowTS")
# map